In [11]:
%cd "/content/drive/MyDrive/Colab Notebooks/3-2/"

/content/drive/MyDrive/Colab Notebooks/3-2


In [12]:
#바이오 기업들 Id 불러와서 리스트로 저장
import pandas as pd

df = pd.read_parquet('./inner_bio_companyId.parquet')
indexes = list(df['companyId'])

In [13]:
#바이오 마켓 데이터 불러오기
import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_market = pd.read_parquet('./market', filters=filters, columns=['companyId', 'pricingDate', 'priceClose', 'adjustmentFactor'])
df_market['companyId'] = list(map(int, df_market['companyId']))
df_market

,companyId,pricingDate,priceClose,adjustmentFactor
0,24153,2023-10-10,34.740000,1.0000000000
1,24153,2023-10-09,34.360000,1.0000000000
2,24153,2023-10-06,34.090000,1.0000000000
3,24153,2023-10-05,34.040000,1.0000000000
4,24153,2023-10-04,33.940000,1.0000000000
...,...,...,...,...
1798012,1844729817,2023-07-26,40.750000,1.0000000000
1798013,1844729817,2023-07-25,39.150000,1.0000000000
1798014,1844729817,2023-07-24,38.700000,1.0000000000
1798015,1844729817,2023-07-21,40.550000,1.0000000000


In [14]:
#바이오 재무 데이터 불러오기

import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_finance = pd.read_parquet('./financials', filters=filters, columns=['companyId', 'periodTypeName', 'fiscalQuarter', 'fiscalYear', 'dataItemName', 'dataItemValue', 'ISOCode', 'unitTypeValue'])

#데이터 처리를 위해 필요한 칼럼들을 Int 형태로 변환
df_finance['companyId'] = list(map(int, df_finance['companyId']))
df_finance['dataItemValue'] = list(map(float, df_finance['dataItemValue']))
df_finance['fiscalYear'] = list(map(int, df_finance['fiscalYear']))
df_finance['unitTypeValue'] = list(map(int, df_finance['unitTypeValue']))

#주어진 UnitTypeValue를 Value에 곱하여 단위를 맞춰줌

df_finance['dataItemValue'] = df_finance['dataItemValue'] * df_finance['unitTypeValue']
df_finance.drop(['unitTypeValue'], axis=1, inplace=True)

##39414143 기업의 데이터가 Outlier라고 판단하여 제거
df_finance = df_finance[df_finance['companyId'] != 39414143]
df_finance

,companyId,periodTypeName,fiscalQuarter,fiscalYear,dataItemName,dataItemValue,ISOCode
0,323656453,Quarterly,2,2021,Capital Expenditure,-19202500.0,TWD
1,323656453,Quarterly,2,2021,Depreciation & Amortization - CF,17492500.0,TWD
2,323656453,Quarterly,2,2021,Change In Inventories,-12314500.0,TWD
3,323656453,Quarterly,2,2021,"Other Investing Activities, Total",-678500.0,TWD
4,323656453,Quarterly,2,2021,Cash from Financing,184408500.0,TWD
...,...,...,...,...,...,...,...
4884277,79723070,Quarterly,2,2021,Earnings From Continuing Operations,-6620576.0,USD
4884278,79723070,Quarterly,2,2021,Net Income - (IS),-6543197.0,USD
4884279,79723070,Quarterly,2,2021,Net Interest Expenses,176798.0,USD
4884280,79723070,Quarterly,2,2021,Net Income to Common Incl Extra Items,-6543197.0,USD


In [15]:
#재무 데이터 중 고유 아이템을 그룹으로 묶어서 분류
financials_unique = df_finance.groupby(['dataItemName', 'periodTypeName']).size().reset_index(name='freq')
financials_unique.drop('freq', axis=1, inplace=True)
financials_unique

,dataItemName,periodTypeName
0,(Gain) Loss From Sale Of Asset,Annual
1,(Gain) Loss From Sale Of Asset,Quarterly
2,(Gain) Loss on Sale of Investments - (CF),Annual
3,(Gain) Loss on Sale of Investments - (CF),Quarterly
4,(Income) Loss On Equity Investments - (CF),Annual
...,...,...
346,Treasury Stock & Other,Quarterly
347,"Unearned Revenue Current, Total",Annual
348,"Unearned Revenue Current, Total",Quarterly
349,Unearned Revenue Non Current,Annual


In [16]:
#한 고유 데이터가 분기별, 연간별로 두 개로 나누어져서 존재함
#그 중에 분기별 데이터만 사용함
quarter = financials_unique[financials_unique['periodTypeName'] == 'Quarterly']
quarter_item = list(quarter['dataItemName'])
quarter_item

['(Gain) Loss From Sale Of Asset',
 '(Gain) Loss on Sale of Investments - (CF)',
 '(Income) Loss On Equity Investments - (CF)',
 'Accounts Payable, Total',
 'Accounts Receivable Long-Term',
 'Accounts Receivable, Total',
 'Accrued Expenses, Total',
 'Accumulated Depreciation',
 'Additional Paid In Capital',
 'Amortization of Deferred Charges, Total - (CF)',
 'Amortization of Goodwill and Intangible Assets - (CF) - (Template Specific)',
 'Amortization of Goodwill and Intangible Assets - (IS)',
 'Asset Writedown',
 'Asset Writedown & Restructuring Costs',
 'Capital Expenditure',
 'Cash Acquisitions',
 'Cash And Equivalents',
 'Cash from Financing',
 'Cash from Investing',
 'Cash from Operations',
 'Change In Accounts Payable',
 'Change In Accounts Receivable',
 'Change In Deferred Taxes',
 'Change In Income Taxes',
 'Change In Inventories',
 'Change in Net Operating Assets',
 'Change in Other Net Operating Assets',
 'Change in Trading Asset Securities',
 'Change in Unearned Revenues',
 '

In [17]:
#재무 데이터가 존재하는 기업이 많이 없는 경우가 있어 데이터 당 기업이 500개 이상인 기업에 대해서만 분석을 진행
financial_item = []
quarter_i = quarter_item.copy()
for item in quarter_i:
  example_feature = df_finance[(df_finance['dataItemName'] == item) & (df_finance['periodTypeName'] == 'Quarterly')]
  id = len(example_feature['companyId'].unique())
  if id > 500:
    financial_item.append(item)

print(len(financial_item))

109


In [18]:
#환율 데이터 불러오기 + 21년 ~ 22년까지 Currency별로 데이터프레임 정리
import pyarrow.parquet as pq
import pandas as pd
df_exchange = pd.read_parquet('./exchange')

df_ex = df_exchange.copy()
df_ex.sort_values('priceDate', ascending=True, inplace=True)
df_ex.set_index('priceDate', drop=True, inplace=True)

code = list(df_ex['ISOCode'].unique())
fx = pd.DataFrame()

for c in code:
  ex = df_ex[(df_ex['ISOCode'] == c)].copy()
  ex.rename(columns={'priceClose' : c}, inplace=True)
  ex.drop(['ISOCode'], axis=1, inplace=True)
  d = ex.loc['2021-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  fx = pd.concat([fx, d], axis=1)

In [19]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리
df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
unique_id = list(df_price['companyId'].unique())
market = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in unique_id:
  ex = df_price[(df_price['companyId'] == id)].copy()
  ex.rename(columns={'price' : id}, inplace=True)
  ex.drop(['companyId'], axis=1, inplace=True)
  ex.sort_values(by='pricingDate', inplace=True)
  ex.set_index('pricingDate', inplace=True, drop=True)
  d = ex.loc['2021-09-22':'2022-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  market = pd.concat([market, d], axis=1)

market2 = market.copy()
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index

#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop(39414143, axis=1, inplace=True)

In [28]:
#한 재무 데이터에 대해 크기 순으로 정렬하여 상위 50%, 하위 50%를 기준으로 롱숏 포트폴리오를 구성 후 수익률 산출
#재무 데이터는 분기별로 나타나기 때문에 예를 들어 2분기 재무 데이터를 통해 3분기 포트폴리오를 구성하는 방향
#따라서 21년 2분기 데이터를 통해 21년 9월 22일 ~ 21년 9월 30일까지의 포트폴리오를 구성
#21년 3분기 데이터를 통해 21년 10월 1일 ~ 21년 12월 31일까지의 포트폴리오를 구성, 이렇게 22년 9월 22일까지 진행
def calculate(item, year, quarter):
  market3 = market2.copy()
  #아이템이랑 연도 받아서 데이터프레임 생성
  example_feature = df_finance[(df_finance['dataItemName'] == item)  & (df_finance['fiscalYear'] == year) & (df_finance['fiscalQuarter'] == quarter) & (df_finance['periodTypeName'] == 'Quarterly')]
  example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  #같은 시점에 똑같은 기업이 똑같은 재무 데이터에 대해 다른 값을 가지고 있다면 평균 내고 하나의 값으로 변환
  #ex) 21년에 A 기업의 영업 이익이 다른 값으로 2~3개가 있음 -> 평균 내서 하나만 남기기

  example2 = example.copy()
  for id in list(map(int, example2['companyId'])):
    num = len(example2[example2['companyId']==id]['dataItemValue'].values)
    if num > 1:
      idx = list(example2[example2['companyId']==id]['dataItemValue'].index)
      avg = []
      for i in idx:
        a = example2.loc[i]['dataItemValue']
        avg.append(a)
      average = sum(avg) / len(avg)
      example2.loc[idx[0]]['dataItemValue'] = average
      example2.drop([i for i in idx[0:]], inplace=True)
  example2.reset_index(drop=True, inplace=True)

  #재무 데이터가 어느 나라의 기업이냐에 따라 명시되어 있는 Currency가 다름 (ISOCode를 통해 확인 가능)
  #21년 2분기 재무 데이터를 활용하여 21년 9월 22일부터 ~ 21년 9월 30일까지의 포트폴리오를 구성하기 때문에,
  #달러로 표시되어 있는게 아니라면 21년 9월 22일의 환율을 기준으로 값을 달러로 변경
  #이런 식으로 각 포트폴리오 구성이 시작되는 날짜를 기준으로 환율을 곱하여 값을 맞춰줌

  if (year == 2021) & (quarter == 2):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-09-22", example2['ISOCode'][i]])
  elif (year == 2021) & (quarter == 3):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-10-01", example2['ISOCode'][i]])
  elif (year == 2021) & (quarter == 4):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-01-01", example2['ISOCode'][i]])
  elif (year == 2022) & (quarter == 1):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-04-01", example2['ISOCode'][i]])
  elif (year == 2022) & (quarter == 2):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-07-01", example2['ISOCode'][i]])

  #작업하는 데이터의 값을 내림차순으로 정렬
  example2.drop('ISOCode', axis=1, inplace=True)
  example2.sort_values(by='dataItemValue', ascending=False, inplace=True)

  #데이터가 있는 기업들의 주가를 불러옴
  comp_id = list(map(int, example2['companyId']))
  market3 = market3.reindex(columns=comp_id)
  market3 = market3.dropna(axis=1)

  #년도에 따라서 포트폴리오를 구성하는 기간이 다르기 때문에 그 구간에서의 주가를 가져옴
  if (year == 2021) & (quarter == 2):
    price_y = market3.loc['2021-09-22':'2021-09-30'].copy()
  elif (year == 2021) & (quarter == 3):
    price_y = market3.loc['2022-10-01':'2022-12-31'].copy()
  elif (year == 2021) & (quarter == 4):
    price_y = market3.loc['2022-01-01':'2022-03-31'].copy()
  elif (year == 2022) & (quarter == 1):
    price_y = market3.loc['2022-04-01':'2022-06-30'].copy()
  elif (year == 2022) & (quarter == 2):
    price_y = market3.loc['2022-07-01':'2022-09-30'].copy()

  for col in price_y.columns:
    price_y[col] = price_y[col].apply(lambda x: float(x))


  #주가를 수익률로 변환 후에 크기 순으로 정렬되어 있기 때문에 상위 50퍼는 롱,
  #하위 50퍼는 숏을 갖는 포트폴리오의 수익률을 새로운 칼럼으로 생성
  price_y = price_y.pct_change(1).dropna()
  price_y = price_y + 1
  cum_ret = price_y.cumprod() - 1
  half = int(len(cum_ret.columns) / 2)
  cum_ret[item] = (cum_ret.iloc[:, 0:half+1].sum(axis=1) - cum_ret.iloc[:, half+1:].sum(axis=1)) / len(cum_ret.columns)

  return cum_ret[item]

In [29]:
import pyarrow.parquet as pq
import pandas as pd
import statsmodels.api as sm

#21년 9월 22일부터 ~ 22년 9월 22일까지 각 예상치에 대해 분기별로 롱숏 포트폴리오를 구성 후 일별 수익률을 계산

pd.options.mode.chained_assignment = None
pair = [[2021, 3], [2021, 4], [2022, 1], [2022, 2]]
multi_agg = pd.DataFrame()
for item in financial_item:
  pf = calculate(item, 2021, 2)
  for y,q in pair:
      temp = calculate(item, y, q)
      pf = pd.concat([pf, temp], axis=0)

  multi_agg = pd.concat([multi_agg, pf], axis=1)

In [39]:
multi_agg

,(Gain) Loss From Sale Of Asset,(Gain) Loss on Sale of Investments - (CF),"Accounts Payable, Total","Accounts Receivable, Total","Accrued Expenses, Total",Accumulated Depreciation,Additional Paid In Capital,"Amortization of Deferred Charges, Total - (CF)",Amortization of Goodwill and Intangible Assets - (CF) - (Template Specific),Asset Writedown,...,Total Debt Issued,Total Debt Repaid,Total Equity,Total Liabilities - (Standard / Utility Template),Total Liabilities And Equity,Total Receivables,Total Revenues,Treasury Stock & Other,"Unearned Revenue Current, Total",Unearned Revenue Non Current
2021-09-23,0.004180,0.005478,0.000196,-0.004863,-0.001422,0.006427,0.001864,0.003315,-0.003944,-0.002669,...,0.000694,-0.002318,0.001047,0.000019,0.000686,-0.002901,-0.002332,-0.001207,0.000505,-0.001782
2021-09-24,-0.018147,-0.004785,-0.004130,-0.007668,-0.005384,0.006336,-0.002845,0.014948,-0.004792,-0.000715,...,-0.004757,0.001307,-0.004937,-0.004815,-0.006688,-0.007089,-0.003087,0.001457,-0.001117,-0.004358
2021-09-27,-0.020428,-0.000964,-0.003964,-0.009806,0.000954,0.011739,0.004205,0.017931,0.004652,0.001560,...,-0.002362,0.009301,-0.001010,-0.002901,-0.001088,-0.006885,0.000500,0.003243,-0.003681,-0.000959
2021-09-28,-0.002016,-0.000173,-0.006742,-0.011683,-0.002508,0.014136,-0.001828,0.020863,-0.001037,-0.003216,...,0.001584,0.021416,-0.005167,-0.006364,-0.005495,-0.010386,-0.002506,0.006167,-0.007925,-0.004476
2021-09-29,-0.003461,-0.003528,-0.012330,-0.013802,-0.006557,0.014381,-0.009171,0.011574,0.001024,-0.026592,...,-0.000256,0.019787,-0.012519,-0.011876,-0.011410,-0.012835,0.000874,0.009885,-0.012297,-0.004474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,-0.054325,0.136794,-0.329185,-0.836782,-0.422505,0.724122,-0.136372,-0.125360,0.035571,-0.141362,...,1.558942,-1.906695,-0.506375,-0.406598,-0.497224,-0.510819,-2.708838,0.184103,-0.087881,-0.296154
2022-09-19,-0.053563,0.140490,-0.343394,-0.892171,-0.453036,0.731081,-0.159508,-0.126655,0.027146,-0.080704,...,1.552752,-1.996903,-0.554611,-0.434829,-0.529572,-0.546842,-2.799785,0.186379,-0.163128,-0.401206
2022-09-20,-0.065579,0.113504,-0.345010,-0.867685,-0.459199,0.737017,-0.172943,-0.128821,0.007933,-0.072819,...,1.611702,-1.912916,-0.555813,-0.443379,-0.530425,-0.530545,-2.744613,0.172720,-0.160342,-0.398767
2022-09-21,-0.057214,0.119995,-0.318360,-0.833973,-0.439902,0.691618,-0.164993,-0.139205,-0.021737,-0.027157,...,1.742164,-1.781743,-0.535512,-0.425658,-0.510368,-0.509269,-2.696750,0.162270,-0.149958,-0.388243


In [31]:
#데이터프레임 생성 과정에서 결측치 존재하면 칼럼 제거
port = multi_agg.copy()
columns_with_missing_values = list(port.columns[port.isnull().any()])
port.drop(columns_with_missing_values, axis=1, inplace=True)
port

,(Gain) Loss From Sale Of Asset,(Gain) Loss on Sale of Investments - (CF),"Accounts Payable, Total","Accounts Receivable, Total","Accrued Expenses, Total",Accumulated Depreciation,Additional Paid In Capital,"Amortization of Deferred Charges, Total - (CF)",Amortization of Goodwill and Intangible Assets - (CF) - (Template Specific),Asset Writedown,...,Total Debt Issued,Total Debt Repaid,Total Equity,Total Liabilities - (Standard / Utility Template),Total Liabilities And Equity,Total Receivables,Total Revenues,Treasury Stock & Other,"Unearned Revenue Current, Total",Unearned Revenue Non Current
2021-09-23,0.004180,0.005478,0.000196,-0.004863,-0.001422,0.006427,0.001864,0.003315,-0.003944,-0.002669,...,0.000694,-0.002318,0.001047,0.000019,0.000686,-0.002901,-0.002332,-0.001207,0.000505,-0.001782
2021-09-24,-0.018147,-0.004785,-0.004130,-0.007668,-0.005384,0.006336,-0.002845,0.014948,-0.004792,-0.000715,...,-0.004757,0.001307,-0.004937,-0.004815,-0.006688,-0.007089,-0.003087,0.001457,-0.001117,-0.004358
2021-09-27,-0.020428,-0.000964,-0.003964,-0.009806,0.000954,0.011739,0.004205,0.017931,0.004652,0.001560,...,-0.002362,0.009301,-0.001010,-0.002901,-0.001088,-0.006885,0.000500,0.003243,-0.003681,-0.000959
2021-09-28,-0.002016,-0.000173,-0.006742,-0.011683,-0.002508,0.014136,-0.001828,0.020863,-0.001037,-0.003216,...,0.001584,0.021416,-0.005167,-0.006364,-0.005495,-0.010386,-0.002506,0.006167,-0.007925,-0.004476
2021-09-29,-0.003461,-0.003528,-0.012330,-0.013802,-0.006557,0.014381,-0.009171,0.011574,0.001024,-0.026592,...,-0.000256,0.019787,-0.012519,-0.011876,-0.011410,-0.012835,0.000874,0.009885,-0.012297,-0.004474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,-0.054325,0.136794,-0.329185,-0.836782,-0.422505,0.724122,-0.136372,-0.125360,0.035571,-0.141362,...,1.558942,-1.906695,-0.506375,-0.406598,-0.497224,-0.510819,-2.708838,0.184103,-0.087881,-0.296154
2022-09-19,-0.053563,0.140490,-0.343394,-0.892171,-0.453036,0.731081,-0.159508,-0.126655,0.027146,-0.080704,...,1.552752,-1.996903,-0.554611,-0.434829,-0.529572,-0.546842,-2.799785,0.186379,-0.163128,-0.401206
2022-09-20,-0.065579,0.113504,-0.345010,-0.867685,-0.459199,0.737017,-0.172943,-0.128821,0.007933,-0.072819,...,1.611702,-1.912916,-0.555813,-0.443379,-0.530425,-0.530545,-2.744613,0.172720,-0.160342,-0.398767
2022-09-21,-0.057214,0.119995,-0.318360,-0.833973,-0.439902,0.691618,-0.164993,-0.139205,-0.021737,-0.027157,...,1.742164,-1.781743,-0.535512,-0.425658,-0.510368,-0.509269,-2.696750,0.162270,-0.149958,-0.388243


In [32]:
#주가 데이터로부터 수익률을 구하고 첫쨰날을 제거
market4 = market2.copy()
market4 = market4.pct_change(1)
market4.drop('2021-09-22', inplace=True)

#주가 수익률과 포트폴리오 수익률을 합쳐주며 특정 날짜에 Null 값이 존재하면 드랍
financial_final = pd.concat([market4, port], axis=1)
a = financial_final.index[financial_final.isnull().any(axis=1)]
financial_final.drop(a, inplace=True)
financial_final

,24153,24246,24366,24572,24816,25192,25594,25605,25622,26241,...,Total Debt Issued,Total Debt Repaid,Total Equity,Total Liabilities - (Standard / Utility Template),Total Liabilities And Equity,Total Receivables,Total Revenues,Treasury Stock & Other,"Unearned Revenue Current, Total",Unearned Revenue Non Current
2021-09-23,0.027715,0.022936,0.011111,0.035703,0.006600,0.014184,-0.001300,0.003416,0.077551,0.000000,...,0.000694,-0.002318,0.001047,0.000019,0.000686,-0.002901,-0.002332,-0.001207,0.000505,-0.001782
2021-09-24,-0.017179,0.062780,-0.011774,0.005327,-0.006696,-0.013986,-0.029278,-0.008827,-0.026515,-0.003279,...,-0.004757,0.001307,-0.004937,-0.004815,-0.006688,-0.007089,-0.003087,0.001457,-0.001117,-0.004358
2021-09-27,-0.000813,-0.002110,-0.018268,0.000000,-0.002341,-0.007092,0.030831,0.000254,0.038911,0.019737,...,-0.002362,0.009301,-0.001010,-0.002901,-0.001088,-0.006885,0.000500,0.003243,-0.003681,-0.000959
2021-09-28,-0.020545,0.012685,-0.041262,0.001559,-0.003942,-0.007143,-0.042263,-0.018316,-0.074906,-0.032258,...,0.001584,0.021416,-0.005167,-0.006364,-0.005495,-0.010386,-0.002506,0.006167,-0.007925,-0.004476
2021-09-29,0.001038,-0.020877,-0.065823,-0.037971,0.012390,-0.021583,-0.023761,-0.005053,0.028340,-0.026667,...,-0.000256,0.019787,-0.012519,-0.011876,-0.011410,-0.012835,0.000874,0.009885,-0.012297,-0.004474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-16,-0.004207,-0.150000,-0.041068,-0.016828,0.015286,-0.113636,0.024942,0.001362,-0.015267,-0.058158,...,1.558942,-1.906695,-0.506375,-0.406598,-0.497224,-0.510819,-2.708838,0.184103,-0.087881,-0.296154
2022-09-19,-0.004608,0.025000,-0.010707,-0.020111,-0.001644,-0.174359,0.032700,-0.005782,-0.054264,-0.022298,...,1.552752,-1.996903,-0.554611,-0.434829,-0.529572,-0.546842,-2.799785,0.186379,-0.163128,-0.401206
2022-09-20,-0.014660,-0.038737,0.010823,0.015721,-0.013217,-0.037267,-0.015464,-0.008438,-0.008197,-0.015789,...,1.611702,-1.912916,-0.555813,-0.443379,-0.530425,-0.530545,-2.744613,0.172720,-0.160342,-0.398767
2022-09-21,-0.032498,-0.074627,-0.072805,-0.035684,-0.014273,-0.141935,-0.023934,-0.021159,-0.016529,-0.044563,...,1.742164,-1.781743,-0.535512,-0.425658,-0.510368,-0.509269,-2.696750,0.162270,-0.149958,-0.388243


In [41]:
list(financial_final.columns)[1158]

'(Gain) Loss From Sale Of Asset'

In [44]:
financial_final.columns[:-109]

Index([     24153,      24246,      24366,      24572,      24816,      25192,
            25594,      25605,      25622,      26241,
       ...
        713710944,  713821465,  716991875, 1671935602, 1673028342, 1676815758,
       1678200093, 1679217092, 1679642292, 1685354464],
      dtype='object', length=1158)

In [ ]:
financial_final.to_csv('financial_final.csv', index=True)

In [ ]:
#Total은 바이오 섹터에 속하는 개별 기업의 개수
total = len(financial_final.columns[:-109])
#Port_col은 각 포트폴리오의 이름을 의미
port_col = list(port.columns)
first_good = []
#개별 기업의 수익률을 y로 두고, 각 데이터에 대한 포트폴리오를 X로 두고 1158개에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in port_col:
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)
  #1159개 중 P-value가 60개 이상인 데이터들을 산출 -> 개별 포트폴리오가 아닌 X를 2개 이상으로 돌리기
  if cnt > 60:
    first_good.append(p)

(Gain) Loss From Sale Of Asset 44
(Gain) Loss on Sale of Investments - (CF) 61
Accounts Payable, Total 44
Accounts Receivable, Total 37
Accrued Expenses, Total 44


In [55]:
tryy = first_good.copy()

[]

In [54]:
pairs = [[first_good[i], first_good[j]] for i in range(len(first_good)) for j in range(i + 1, len(first_good))]
pairs

[]

In [51]:
pairs = [[first_good[i], first_good[j]] for i in range(len(first_good)) for j in range(i + 1, len(first_good))]
total = len(financial_final.columns[:-109])
first_good = []
#개별 기업의 수익률을 y로 두고, X1, X2와 1158개의 y에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in pairs:
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)

In [ ]:
port[['Asset Writedown & Restructuring Costs', 'Sale (Purchase) of Intangible assets', 'Long-Term Debt', 'Other Unusual Items']].corr()

,Asset Writedown & Restructuring Costs,Sale (Purchase) of Intangible assets,Long-Term Debt,Other Unusual Items
Asset Writedown & Restructuring Costs,1.000000,-0.974086,-0.132070,0.471776
Sale (Purchase) of Intangible assets,-0.974086,1.000000,0.024891,-0.412858
Long-Term Debt,-0.132070,0.024891,1.000000,-0.045171
Other Unusual Items,0.471776,-0.412858,-0.045171,1.000000
